In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
from torch import nn, optim
import torch.nn.functional as F
import tqdm
from tqdm import tnrange
import matplotlib.pyplot as pl
# Load the data
#df_test = pd.read_csv('./datasets/test.csv')
#'C:\Users\nuppa\Downloads\ames.csv'
X_train_non = pd.read_csv('./sample_data/ames_X_train.csv')
y_train_non = pd.read_csv('./sample_data/ames_y_train.csv')
#data = pd.read_csv(r'C:\Users\nuppa\Downloads\ames.csv')

X_train_non.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Lot.Area       2929 non-null   int64
 1   Year.Built     2929 non-null   int64
 2   X1st.Flr.SF    2929 non-null   int64
 3   X2nd.Flr.SF    2929 non-null   int64
 4   Gr.Liv.Area    2929 non-null   int64
 5   Full.Bath      2929 non-null   int64
 6   Half.Bath      2929 non-null   int64
 7   Bedroom.AbvGr  2929 non-null   int64
 8   Kitchen.AbvGr  2929 non-null   int64
 9   TotRms.AbvGrd  2929 non-null   int64
 10  Fireplaces     2929 non-null   int64
 11  Garage.Area    2929 non-null   int64
 12  Pool.Area      2929 non-null   int64
dtypes: int64(13)
memory usage: 297.6 KB


In [ ]:
#train_tensor = torch.tensor(train.values)

print(X_train_non.shape)
print(y_train_non.shape)

X_train_all = torch.tensor(X_train_non.values).float()
y_train_all = torch.tensor(y_train_non.values).float()
X_train = X_train_all[:1829]
y_train = y_train_all[:1829]
X_test = X_train_all[1829:]
y_test = y_train_all[1829:]
print(X_train)
print(y_train)
print(X_train.shape)
print(y_train.shape)



(2929, 13)
(2929, 1)
tensor([[3.1770e+04, 1.9600e+03, 1.6560e+03,  ..., 2.0000e+00, 5.2800e+02,
         0.0000e+00],
        [1.1622e+04, 1.9610e+03, 8.9600e+02,  ..., 0.0000e+00, 7.3000e+02,
         0.0000e+00],
        [1.4267e+04, 1.9580e+03, 1.3290e+03,  ..., 0.0000e+00, 3.1200e+02,
         0.0000e+00],
        ...,
        [6.1730e+03, 1.9670e+03, 9.0200e+02,  ..., 0.0000e+00, 2.8800e+02,
         0.0000e+00],
        [1.9522e+04, 1.9900e+03, 1.2710e+03,  ..., 1.0000e+00, 5.6400e+02,
         0.0000e+00],
        [1.7542e+04, 1.9740e+03, 1.5160e+03,  ..., 2.0000e+00, 5.1800e+02,
         0.0000e+00]])
tensor([[215000.],
        [105000.],
        [172000.],
        ...,
        [125500.],
        [300000.],
        [294000.]])
torch.Size([1829, 13])
torch.Size([1829, 1])


In [ ]:
print(X_test)
print(y_test)
print(X_test.shape)
print(y_test.shape)


tensor([[1.0751e+04, 1.9740e+03, 1.0370e+03,  ..., 0.0000e+00, 4.3100e+02,
         0.0000e+00],
        [1.6647e+04, 1.9750e+03, 1.7010e+03,  ..., 2.0000e+00, 6.1100e+02,
         0.0000e+00],
        [1.2712e+04, 1.9730e+03, 1.0550e+03,  ..., 1.0000e+00, 5.4200e+02,
         0.0000e+00],
        ...,
        [1.0441e+04, 1.9920e+03, 9.7000e+02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0010e+04, 1.9740e+03, 1.3890e+03,  ..., 1.0000e+00, 4.1800e+02,
         0.0000e+00],
        [9.6270e+03, 1.9930e+03, 9.9600e+02,  ..., 1.0000e+00, 6.5000e+02,
         0.0000e+00]])
tensor([[129000.],
        [171000.],
        [147000.],
        ...,
        [132000.],
        [170000.],
        [188000.]])
torch.Size([1100, 13])
torch.Size([1100, 1])


In [ ]:
##X_train = 1465
##Y_Train = 1465
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(13, 13)

        self.fc2 = nn.Linear(144,72)
        self.fc3 = nn.Linear(72,18)
        self.fc4 = nn.Linear(18, 1)

        self.layer_extend = nn.Linear(13, 144)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = self.layer_extend(x)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
     
        return x

In [ ]:
model = Regressor()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 300


train_losses, val_losses = [], []

for e in range(epochs):
    model.train()
    train_loss = 0
    val_loss_1=0
    val_loss_sum = 0
    for i in range(len(X_train)):
        optimizer.zero_grad()
        output = model(X_train[i])
     
        loss = criterion(output, y_train[i])
      
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
      
        
        
        
    print("Epoch :", e, "train_loss :", train_loss/len(X_train), )    
       
    train_losses.append(train_loss/len(X_train))

Epoch : 0 train_loss : 9913848166.161205
Epoch : 1 train_loss : 5966319127.582819
Epoch : 2 train_loss : 5079720537.527192
Epoch : 3 train_loss : 5075030544.799404
Epoch : 4 train_loss : 5088571526.275908
Epoch : 5 train_loss : 4588048255.70863
Epoch : 6 train_loss : 4650100247.311468
Epoch : 7 train_loss : 4786279085.58304
Epoch : 8 train_loss : 4778835697.55613
Epoch : 9 train_loss : 4473481264.075562
Epoch : 10 train_loss : 4737842001.078693
Epoch : 11 train_loss : 4006771853.7823157
Epoch : 12 train_loss : 4166935985.322684
Epoch : 13 train_loss : 4514786236.879405
Epoch : 14 train_loss : 4399792084.098866
Epoch : 15 train_loss : 4859795398.271276
Epoch : 16 train_loss : 4567831188.331258
Epoch : 17 train_loss : 4562342946.002263
Epoch : 18 train_loss : 4284893960.481242
Epoch : 19 train_loss : 4295720635.994829
Epoch : 20 train_loss : 4357633665.872698


KeyboardInterrupt: ignored

In [ ]:

for i in range(len(X_test)):
    output = model(X_test[i])
    target = y_test[i]
    
    
      
        
        
        
    print("Model Prediction :",  output ,                  "Actual Price", y_test[i] )    
       


Model Prediction : tensor([147010.9531], grad_fn=<AddBackward0>) Actual Price tensor([129000.])
Model Prediction : tensor([232820.9688], grad_fn=<AddBackward0>) Actual Price tensor([171000.])
Model Prediction : tensor([161508.5469], grad_fn=<AddBackward0>) Actual Price tensor([147000.])
Model Prediction : tensor([325229.7812], grad_fn=<AddBackward0>) Actual Price tensor([302000.])
Model Prediction : tensor([361596.9688], grad_fn=<AddBackward0>) Actual Price tensor([285000.])
Model Prediction : tensor([189971.4531], grad_fn=<AddBackward0>) Actual Price tensor([214000.])
Model Prediction : tensor([157784.2344], grad_fn=<AddBackward0>) Actual Price tensor([166000.])
Model Prediction : tensor([214840.2344], grad_fn=<AddBackward0>) Actual Price tensor([193800.])
Model Prediction : tensor([178329.7969], grad_fn=<AddBackward0>) Actual Price tensor([208900.])
Model Prediction : tensor([179629.1562], grad_fn=<AddBackward0>) Actual Price tensor([194000.])
Model Prediction : tensor([212071.], gra

In [ ]:
accuracy = [np.zeros(len(X_test))]
print(y_test[0:10])


tensor([[129000.],
        [171000.],
        [147000.],
        [302000.],
        [285000.],
        [214000.],
        [166000.],
        [193800.],
        [208900.],
        [194000.]])
